In [1]:
import cv2
import numpy as np

In [2]:
#setting
colorGreen = (0,255,0)
scale = 0.75
img_size = (320,320)
confThreshold = 0.5 # ค่า Threshlod
nmsThreshold = 0.4 # ค่า  NMS Threshold

#setting video
# cap = cv2.VideoCapture() #web cam ,vdo capture
# cap.set(3,640) # width
# cap.set(4,480) #height

cap = cv2.VideoCapture('vdol.mp4') # หากใช้กล่อง web cam ให้ comment บันทัดนี้

if(not(cap.isOpened())):
    print("Could not open the VDO file")
else:
    print("Open VDO succeses!")

Open VDO succeses!


In [3]:
classesFile = 'coco.names'
classNames = []
with open("coco.names","r") as f:
    classNames = [line.strip() for line in f.readlines()]
print("dataclass",len(classNames),"class")

dataclass 80 class


In [4]:
#config
modelConfiguration = 'yolov3.cfg'
modelWeights = 'yolov3.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration,modelWeights)
layerNames = net.getLayerNames()
outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
#bbox
def locateObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]* wT),int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2),int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))
        indices = cv2.dnn.NMSBoxes(bbox,confs,confThreshold,nmsThreshold)

        for i in indices:
            i = i-1
            box = bbox[i]
            x,y,w,h = box[0],box[1],box[2],box[3]
            cv2.rectangle(img,(x,y),(x+w,y+h),colorGreen,2)
            label = str(classNames[classIds[i]])
            label = '{} {:.1f}%'.format(label,confs[i]*100)
            cv2.putText(img,label,(x,y-8),cv2.FONT_HERSHEY_SIMPLEX,0.6,colorGreen,2)

In [ ]:
# Main Loop
while (cap.isOpened()):
    success,img = cap.read()
    img = cv2.resize(img,None,fx=scale,fy=scale,interpolation=cv2.INTER_AREA) # หากใช้กล่อง web cam ให้ comment บันทัดนี้
    blob = cv2.dnn.blobFromImage(img,0.00392,img_size,(0,0,0),crop=False)
    # 0.00392 = 1/255
    net.setInput(blob) #Detect
    outputs = net.forward(outputNames)
    locateObjects(outputs,img) #bbox
    
    cv2.imshow('Mywindow',img)
    if cv2.waitKey(1) == 27: # ESC to exit
        break
cap.release() # เครียร์ object
cv2.destroyAllWindows()